# Flight seats downloader

Data taken from [US Bureau of Transportation](https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236)

Field descriptions provided [here](https://www.transtats.bts.gov/Fields.asp?Table_ID=311)

In [1]:
import requests
from selenium import webdriver


In [2]:
chrome_options = webdriver.ChromeOptions()
prefs = {'download.default_directory' : '/Users/jaromeleslie/Documents/MDS/Personal_projects/Ohare_taxi_demand/data/seats'}
chrome_options.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(chrome_options=chrome_options)
driver.get('https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=311')



/Users/jaromeleslie/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: use options instead of chrome_options
  """


In [3]:
targ_years= list(range(2013,2020,1))
targ_years = list(map(str,targ_years))

In [4]:
#STEP 1. LOCATE DOWNLOAD BUTTON

download_bt = driver.find_element_by_xpath('//*[@id="content"]/table[1]/tbody/tr/td[2]/table[3]/tbody/tr/td[2]/button[1]')
# download_bt.click()

In [6]:
#STEP 2. SELECT FIELDS OF INTEREST (IGNORING DEFAULTS)

# DEPARTURES SCHEDULED
dep_sched_sel = driver.find_element_by_xpath('/html/body/div[3]/div[3]/table[1]/tbody/tr/td[2]/table[4]/tbody/tr[3]/td[1]/input')
dep_sched_sel.click()

# DEPARTURES PERFORMED
dep_perf_sel = driver.find_element_by_xpath('/html/body/div[3]/div[3]/table[1]/tbody/tr/td[2]/table[4]/tbody/tr[4]/td[1]/input')
dep_perf_sel.click()


# SEATS
seats_sel = driver.find_element_by_xpath('/html/body/div[3]/div[3]/table[1]/tbody/tr/td[2]/table[4]/tbody/tr[6]/td[1]/input')
seats_sel.click()

# PASSENGERS
pass_sel = driver.find_element_by_xpath('/html/body/div[3]/div[3]/table[1]/tbody/tr/td[2]/table[4]/tbody/tr[7]/td[1]/input')
pass_sel.click()



In [10]:
#STEP 3. LOOP OVER YEARS OF INTEREST

#FIND DROPDOWN FOR SELECTABLE YEARS 
year_sel = driver.find_element_by_id("XYEAR")
all_years = year_sel.find_elements_by_tag_name("option")


#OUTER LOOP FOR EACH YEAR
for year in all_years:
    if year.get_attribute("value") in targ_years:
        print("Value is: %s" % year.get_attribute("value"))
        year.click()
        

        #EXECUTE DOWNLOAD
        download_bt.click()


Value is: 2013
Value is: 2014
Value is: 2015
Value is: 2016
Value is: 2017
Value is: 2018
Value is: 2019


# Merge downloads into single file

In [9]:
#STARTING WITH 84 ZIPFILES, MAKE ORD_OTP.CSV

import pandas as pd
import numpy as np
import zipfile as zp

import requests
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import pytest

In [11]:
entries = []

for i in range(72,84,1):
    if i == 0:
        with zp.ZipFile('../data/seats/691436399_T_T100D_SEGMENT_ALL_CARRIER.zip') as myzip:
            myzip.extract('691436399_T_T100D_SEGMENT_ALL_CARRIER.csv',path='../data/seats')
            df = pd.read_csv('../data/seats/691436399_T_T100D_SEGMENT_ALL_CARRIER.csv')
            entries.append(df.query('DEST == "ORD"'))
            
    else:
        with zp.ZipFile('../data/otp/1051953426_T_ONTIME_REPORTING '+'('+str(i)+').zip') as myzip:
            myzip.extract('1051953426_T_ONTIME_REPORTING.csv', path ='../data/otp')
            df = pd.read_csv('../data/otp/1051953426_T_ONTIME_REPORTING.csv')
            entries.append(df.query('DEST == "ORD"'))

In [12]:
combined_ord_seats = pd.concat(entries)

In [13]:
combined_ord_seats.head()

,DAY_OF_WEEK,FL_DATE,OP_UNIQUE_CARRIER,TAIL_NUM,OP_CARRIER_FL_NUM,ORIGIN_AIRPORT_ID,ORIGIN_AIRPORT_SEQ_ID,ORIGIN_CITY_MARKET_ID,ORIGIN,DEST_AIRPORT_ID,DEST_AIRPORT_SEQ_ID,DEST_CITY_MARKET_ID,DEST,ARR_TIME,ARR_DELAY,Unnamed: 15
3820,1,2019-01-07,9E,N933XJ,5408,12478,1247805,31703,JFK,13930,1393007,30977,ORD,1659.0,-30.0,NaN
3821,1,2019-01-14,9E,N676CA,5408,12478,1247805,31703,JFK,13930,1393007,30977,ORD,1659.0,-30.0,NaN
3822,1,2019-01-21,9E,N181PQ,5408,12478,1247805,31703,JFK,13930,1393007,30977,ORD,1838.0,69.0,NaN
3823,1,2019-01-28,9E,N313PQ,5408,12478,1247805,31703,JFK,13930,1393007,30977,ORD,NaN,NaN,NaN
3927,1,2019-01-07,9E,N310PQ,5433,12478,1247805,31703,JFK,13930,1393007,30977,ORD,948.0,-14.0,NaN
